In [5]:
import random

class WumpusWorld:
    def __init__(self, size, max_moves):
        self.size = size
        self.max_moves = max_moves
        self.grid = [[Cell() for _ in range(size)] for _ in range(size)]
        self.agent_pos = (size - 1, 0)  # Starting from bottom left
        self.agent_direction = "right"  # Starting direction
        self.knowledge_base = [[None for _ in range(size)] for _ in range(size)]
        self.move_stack = []
        self.move_count = 0
        self.score = 0
        self.arrow_used = False
        self.wumpus_alive = True
        self.generate_world()

    def generate_world(self):
        wumpus_x, wumpus_y = self.random_position()
        self.grid[wumpus_x][wumpus_y].has_wumpus = True
        num_pits = min(3, self.size * self.size - 1)  # Limiting the number of pits to 3
        for _ in range(num_pits):
            pit_x, pit_y = self.random_position()
            self.grid[pit_x][pit_y].has_pit = True
            # Mark adjacent cells as having breeze
            for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                new_x, new_y = pit_x + dx, pit_y + dy
                if (new_x, new_y) == self.agent_pos:
                    continue  # Skip marking the starting cell
                if 0 <= new_x < self.size and 0 <= new_y < self.size:
                    self.grid[new_x][new_y].has_breeze = True
        gold_x, gold_y = self.random_position()
        self.grid[gold_x][gold_y].has_gold = True
        for i in range(self.size):
            for j in range(self.size):
                if self.grid[i][j].has_wumpus:
                    for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                        if 0 <= i + dx < self.size and 0 <= j + dy < self.size:
                            self.grid[i + dx][j + dy].has_stench = True
                if self.grid[i][j].has_gold:
                    self.grid[i][j].has_glitter = True

    def random_position(self):
        x = random.randint(0, self.size - 1)
        y = random.randint(0, self.size - 1)
        if (x, y) == (self.size - 1, 0):
            return self.random_position()  # Ensure the random position is not the start
        return x, y

    def print_world(self):
        print(f"\nMove {self.move_count}: Score = {self.score}")
        for i in range(self.size):
            for j in range(self.size):
                cell = self.grid[i][j]
                if (i, j) == self.agent_pos:
                    if self.agent_direction == "up":
                        print("↑A", end="\t")  # A for Agent, ↑ for direction up
                    elif self.agent_direction == "down":
                        print("↓A", end="\t")  # ↓ for direction down
                    elif self.agent_direction == "left":
                        print("←A", end="\t")  # ← for direction left
                    else:
                        print("→A", end="\t")  # → for direction right
                else:
                    symbols = ""
                    if cell.has_wumpus:
                        symbols += "👹"  # 👹 for Wumpus
                    if cell.has_pit:
                        symbols += "🕳️"  # 🕳️ for Pit
                    if cell.has_gold:
                        symbols += "💰"  # 💰 for Gold
                    if cell.has_stench:
                        symbols += "💨"  # 💨 for Stench
                    if cell.has_breeze:
                        symbols += "🍃"  # 🍃 for Breeze
                    if cell.has_glitter:
                        symbols += "✨"  # ✨ for Glitter (Gold nearby)
                    if symbols:
                        print(symbols, end="\t")
                    else:
                        print("⬜", end="\t")  # ⬜ for an empty cell
            print("\n")

    def sense_environment(self):
        x, y = self.agent_pos
        current_cell = self.grid[x][y]
        self.knowledge_base[x][y] = "safe"  # Mark the current cell as safe
        current_cell.visited = True

        # Sense the environment
        percepts = [None] * 5  # [Stench, Breeze, Glitter, Bump, Scream]
        if current_cell.has_breeze:
            percepts[1] = "Breeze"
            self.mark_adjacent_as_potential("pit", x, y)
        if current_cell.has_stench:
            percepts[0] = "Stench"
            self.mark_adjacent_as_potential("wumpus", x, y)
        if current_cell.has_glitter:
            percepts[2] = "Glitter"

        print(f"Percepts: {percepts}")

    def mark_adjacent_as_potential(self, danger, x, y):
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < self.size and 0 <= new_y < self.size:
                if self.knowledge_base[new_x][new_y] is None:
                    self.knowledge_base[new_x][new_y] = danger

    def decide_next_move(self):
        x, y = self.agent_pos
        possible_moves = []

        # First, try to move to adjacent safe and unvisited cells
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < self.size and 0 <= new_y < self.size:
                if self.knowledge_base[new_x][new_y] == "safe" and not self.grid[new_x][new_y].visited:
                    possible_moves.append((new_x, new_y))

        if possible_moves:
            return random.choice(possible_moves)

        # If no safe unvisited cells, move to any unvisited cell
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < self.size and 0 <= new_y < self.size:
                if not self.grid[new_x][new_y].visited:
                    return new_x, new_y

        # If all adjacent cells are visited, backtrack using the stack
        if self.move_stack:
            return self.move_stack.pop()

        return None

    def play(self):
        while self.move_count < self.max_moves:
            self.sense_environment()
            next_move = self.decide_next_move()
            if not next_move:
                print("No safe moves left. Agent stopped.")
                break

            # Save the current position to the move stack for backtracking
            self.move_stack.append(self.agent_pos)
            self.agent_pos = next_move
            self.move_count += 1
            self.score -= 1  # Penalty for each move
            self.print_world()

            # Check if agent has found the gold
            if self.grid[self.agent_pos[0]][self.agent_pos[1]].has_gold:
                print("Congratulations! You found the gold!")
                self.score += 1000  # Reward for finding the gold
                break

            # Check if agent has fallen into a pit or encountered the Wumpus
            if self.grid[self.agent_pos[0]][self.agent_pos[1]].has_pit:
                print("You fell into a pit! Game over.")
                self.score -= 1000  # Penalty for falling into a pit
                break
            if self.grid[self.agent_pos[0]][self.agent_pos[1]].has_wumpus and self.wumpus_alive:
                print("You were eaten by the Wumpus! Game over.")
                self.score -= 1000  # Penalty for being eaten by the Wumpus
                break

        if self.grid[self.agent_pos[0]][self.agent_pos[1]].has_gold:
            print("The agent successfully retrieved the gold and exited the cave.")
        else:
            print("Out of moves! The gold was not found.")
        print(f"Final Score: {self.score}")

class Cell:
    def __init__(self):
        self.has_wumpus = False
        self.has_pit = False
        self.has_gold = False
        self.has_stench = False
        self.has_breeze = False
        self.has_glitter = False
        self.visited = False

if __name__ == "__main__":
    size = 4  # 4x4 grid
    max_moves = int(input("Enter the number of moves: "))
    game = WumpusWorld(size, max_moves)
    game.print_world()  # Show initial world state
    game.play()


Enter the number of moves:  5



Move 0: Score = 0
🕳️🍃	🍃	⬜	🍃	

🕳️🍃	🍃	🍃	🕳️	

🍃	💰✨	⬜	💨🍃	

→A	⬜	💨	👹	

Percepts: [None, None, None, None, None]

Move 1: Score = -1
🕳️🍃	🍃	⬜	🍃	

🕳️🍃	🍃	🍃	🕳️	

🍃	💰✨	⬜	💨🍃	

⬜	→A	💨	👹	

Percepts: [None, None, None, None, None]

Move 2: Score = -2
🕳️🍃	🍃	⬜	🍃	

🕳️🍃	🍃	🍃	🕳️	

🍃	💰✨	⬜	💨🍃	

⬜	⬜	→A	👹	

Percepts: ['Stench', None, None, None, None]

Move 3: Score = -3
🕳️🍃	🍃	⬜	🍃	

🕳️🍃	🍃	🍃	🕳️	

🍃	💰✨	⬜	💨🍃	

⬜	⬜	💨	→A	

You were eaten by the Wumpus! Game over.
Out of moves! The gold was not found.
Final Score: -1003


In [10]:
import os
import random
import time

class WumpusWorld:
    def __init__(self, size=4):
        self.size = size
        self.environment = [[' ' for _ in range(size)] for _ in range(size)]
        self.player_pos = [size-1, 0]  # Start at the bottom-left corner
        self.wumpus_pos = [random.randint(0, size-2), random.randint(1, size-1)]
        self.pit_pos = [random.randint(0, size-2), random.randint(1, size-1)]
        self.gold_pos = [random.randint(0, size-1), random.randint(1, size-1)]
        self.visited = set()
        self.safe_zones = set()
        self.unsafe_zones = set()
        self.moves = 0

    def clear_screen(self):
        os.system('cls' if os.name == 'nt' else 'clear')

    def print_environment(self):
        self.clear_screen()
        print("Wumpus World:")
        print("-------------")
        for i in range(self.size):
            row = ""
            for j in range(self.size):
                cell_content = ""
                if [i, j] == self.player_pos:
                    cell_content += "A"  # Agent
                else:
                    percepts = self.get_percepts_at([i, j])
                    if percepts["glitter"]:
                        cell_content += "G"  # Glitter (Gold)
                    elif [i, j] == self.wumpus_pos:
                        cell_content += "W"  # Wumpus
                    elif [i, j] == self.pit_pos:
                        cell_content += "P"  # Pit
                    else:
                        if percepts["stench"]:
                            cell_content += "S"  # Stench
                        if percepts["breeze"]:
                            cell_content += "B"  # Breeze
                        if not cell_content:
                            cell_content = "."

                row += f" {cell_content} "
            print(row)
        print("-------------\n")

    def get_percepts_at(self, position):
        percepts = {"breeze": False, "stench": False, "glitter": False}
        x, y = position

        if position == self.gold_pos:
            percepts["glitter"] = True

        if any([x, y] == [self.wumpus_pos[0] + dx, self.wumpus_pos[1] + dy] for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]):
            percepts["stench"] = True

        if any([x, y] == [self.pit_pos[0] + dx, self.pit_pos[1] + dy] for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]):
            percepts["breeze"] = True

        return percepts

    def get_percepts(self):
        return self.get_percepts_at(self.player_pos)

    def move_player(self, direction):
        x, y = self.player_pos
        if direction == 'up' and x > 0:
            self.player_pos = [x-1, y]
        elif direction == 'down' and x < self.size-1:
            self.player_pos = [x+1, y]
        elif direction == 'left' and y > 0:
            self.player_pos = [x, y-1]
        elif direction == 'right' and y < self.size-1:
            self.player_pos = [x, y+1]
        else:
            print("Invalid move or out of bounds.")
            return False

        self.moves += 1
        return True

    def update_knowledge_base(self):
        self.visited.add(tuple(self.player_pos))
        percepts = self.get_percepts()
        
        if not percepts["breeze"] and not percepts["stench"]:
            self.safe_zones.update(self.get_adjacent_cells(self.player_pos))
        
        if percepts["breeze"]:
            self.unsafe_zones.update(self.get_adjacent_cells(self.player_pos))
        
        if percepts["stench"]:
            self.unsafe_zones.update(self.get_adjacent_cells(self.player_pos))

    def get_adjacent_cells(self, position):
        x, y = position
        adjacent_cells = []
        if x > 0:
            adjacent_cells.append((x-1, y))
        if x < self.size - 1:
            adjacent_cells.append((x+1, y))
        if y > 0:
            adjacent_cells.append((x, y-1))
        if y < self.size - 1:
            adjacent_cells.append((x, y+1))
        return adjacent_cells

    def choose_next_move(self):
        for cell in self.safe_zones:
            if cell not in self.visited:
                x, y = cell
                px, py = self.player_pos
                if x < px:
                    return "up"
                elif x > px:
                    return "down"
                elif y < py:
                    return "left"
                elif y > py:
                    return "right"
        return None

    def check_status(self):
        if self.player_pos == self.wumpus_pos:
            print("Player encountered the Wumpus. Game Over!")
            return False
        elif self.player_pos == self.pit_pos:
            print("Player fell into a pit. Game Over!")
            return False
        elif self.player_pos == self.gold_pos:
            print("Player found the gold. You Win!")
            return False
        return True

def play_game():
    wumpus_world = WumpusWorld()
    print("Welcome to the Wumpus World!")
    num_moves = int(input("Enter the number of moves: "))

    while wumpus_world.moves < num_moves:
        wumpus_world.print_environment()
        wumpus_world.update_knowledge_base()

        move = wumpus_world.choose_next_move()
        if move is None:
            print("No safe moves available. Game Over!")
            break

        print(f"AI decided to move {move}.")
        if not wumpus_world.move_player(move):
            continue

        if not wumpus_world.check_status():
            break

        time.sleep(1)  # Pause to simulate movement and give a clearer step-by-step display

    if wumpus_world.moves == num_moves:
        print("Out of moves! Game Over.")

if __name__ == "__main__":
    play_game()


Welcome to the Wumpus World!


Enter the number of moves:  4


Wumpus World:
-------------
 .  .  S  W 
 .  .  G  S 
 .  B  P  B 
 A  .  B  . 
-------------

AI decided to move right.
Wumpus World:
-------------
 .  .  S  W 
 .  .  G  S 
 .  B  P  B 
 .  A  B  . 
-------------

AI decided to move up.
Wumpus World:
-------------
 .  .  S  W 
 .  .  G  S 
 .  A  P  B 
 .  .  B  . 
-------------

AI decided to move left.
Wumpus World:
-------------
 .  .  S  W 
 .  .  G  S 
 A  B  P  B 
 .  .  B  . 
-------------

AI decided to move up.
Out of moves! Game Over.


In [1]:
import os
import random
import time

class WumpusWorld:
    def __init__(self, size=4):
        self.size = size
        self.environment = [[' ' for _ in range(size)] for _ in range(size)]
        self.player_pos = [size-1, 0]  # Start at the bottom-left corner
        self.wumpus_pos = [random.randint(0, size-2), random.randint(1, size-1)]
        self.pit_pos = [random.randint(0, size-2), random.randint(1, size-1)]
        self.gold_pos = [random.randint(0, size-1), random.randint(1, size-1)]
        self.visited = set()
        self.safe_zones = set()
        self.unsafe_zones = set()
        self.moves = 0

    def clear_screen(self):
        os.system('cls' if os.name == 'nt' else 'clear')

    def print_environment(self):
        self.clear_screen()
        print("Wumpus World:")
        print("-------------")
        for i in range(self.size):
            row = ""
            for j in range(self.size):
                cell_content = self.get_cell_content([i, j])
                row += f" {cell_content} "
            print(row)
        print("-------------\n")

    def get_cell_content(self, position):
        if position == self.player_pos:
            return "A"  # Agent
        elif position == self.gold_pos:
            return "G"  # Glitter (Gold)
        elif position == self.wumpus_pos:
            return "W"  # Wumpus
        elif position == self.pit_pos:
            return "P"  # Pit
        else:
            percepts = self.get_percepts_at(position)
            if percepts["glitter"]:
                return "G"
            elif percepts["stench"]:
                return "S"  # Stench
            elif percepts["breeze"]:
                return "B"  # Breeze
            else:
                return "."

    def get_percepts_at(self, position):
        x, y = position
        percepts = {"breeze": False, "stench": False, "glitter": False}

        if position == self.gold_pos:
            percepts["glitter"] = True

        # Check for stench
        if self.is_adjacent(position, self.wumpus_pos):
            percepts["stench"] = True

        # Check for breeze
        if self.is_adjacent(position, self.pit_pos):
            percepts["breeze"] = True

        return percepts

    def is_adjacent(self, pos1, pos2):
        x1, y1 = pos1
        x2, y2 = pos2
        return abs(x1 - x2) + abs(y1 - y2) == 1

    def get_percepts(self):
        return self.get_percepts_at(self.player_pos)

    def move_player(self, direction):
        x, y = self.player_pos
        if direction == 'up' and x > 0:
            self.player_pos = [x-1, y]
        elif direction == 'down' and x < self.size-1:
            self.player_pos = [x+1, y]
        elif direction == 'left' and y > 0:
            self.player_pos = [x, y-1]
        elif direction == 'right' and y < self.size-1:
            self.player_pos = [x, y+1]
        else:
            print("Invalid move or out of bounds.")
            return False

        self.moves += 1
        return True

    def update_knowledge_base(self):
        self.visited.add(tuple(self.player_pos))
        percepts = self.get_percepts()
        
        if not percepts["breeze"] and not percepts["stench"]:
            self.safe_zones.update(self.get_adjacent_cells(self.player_pos))
        
        if percepts["breeze"] or percepts["stench"]:
            self.unsafe_zones.update(self.get_adjacent_cells(self.player_pos))

    def get_adjacent_cells(self, position):
        x, y = position
        adjacent_cells = []
        if x > 0:
            adjacent_cells.append((x-1, y))
        if x < self.size - 1:
            adjacent_cells.append((x+1, y))
        if y > 0:
            adjacent_cells.append((x, y-1))
        if y < self.size - 1:
            adjacent_cells.append((x, y+1))
        return adjacent_cells

    def choose_next_move(self):
        for cell in self.safe_zones:
            if cell not in self.visited:
                return self.get_move_direction(cell)
        return None

    def get_move_direction(self, destination):
        x, y = destination
        px, py = self.player_pos
        if x < px:
            return "up"
        elif x > px:
            return "down"
        elif y < py:
            return "left"
        elif y > py:
            return "right"
        return None

    def check_status(self):
        if self.player_pos == self.wumpus_pos:
            print("Player encountered the Wumpus. Game Over!")
            return False
        elif self.player_pos == self.pit_pos:
            print("Player fell into a pit. Game Over!")
            return False
        elif self.player_pos == self.gold_pos:
            print("Player found the gold. You Win!")
            return False
        return True

def play_game():
    wumpus_world = WumpusWorld()
    print("Welcome to the Wumpus World!")
    num_moves = int(input("Enter the number of moves: "))

    while wumpus_world.moves < num_moves:
        wumpus_world.print_environment()
        wumpus_world.update_knowledge_base()

        move = wumpus_world.choose_next_move()
        if move is None:
            print("No safe moves available. Game Over!")
            break

        print(f"AI decided to move {move}.")
        if not wumpus_world.move_player(move):
            continue

        if not wumpus_world.check_status():
            break

        time.sleep(1)  # Pause to simulate movement and give a clearer step-by-step display

    if wumpus_world.moves == num_moves:
        print("Out of moves! Game Over.")

if __name__ == "__main__":
    play_game()


Welcome to the Wumpus World!


Enter the number of moves:  4


Wumpus World:
-------------
 .  .  B  P 
 .  .  .  S 
 .  .  S  W 
 A  .  .  G 
-------------

AI decided to move right.
Wumpus World:
-------------
 .  .  B  P 
 .  .  .  S 
 .  .  S  W 
 .  A  .  G 
-------------

AI decided to move up.
Wumpus World:
-------------
 .  .  B  P 
 .  .  .  S 
 .  A  S  W 
 .  .  .  G 
-------------

AI decided to move up.
Wumpus World:
-------------
 .  .  B  P 
 .  A  .  S 
 .  .  S  W 
 .  .  .  G 
-------------

AI decided to move up.
Out of moves! Game Over.
